# 第3章: JuMPによる線形計画法の応用 (Julia版)

この章では、JuMPを使ってより実践的な線形計画問題を解きます。

In [ ]:
using JuMP
using HiGHS

## 例題1: ブレンド問題（飼料配合）

### 問題

2種類の原料（A, B）を混ぜて飼料を作ります。飼料は最低限の栄養素を含む必要があります。

|  | コスト | タンパク質 | 脂肪 | 繊維 |
|--|--------|------------|------|------|
| 原料A | \$5/kg | 30% | 10% | 5% |
| 原料B | \$8/kg | 20% | 15% | 10% |

**必要量:**
- タンパク質: 最低25%
- 脂肪: 最低12%
- 繊維: 最大8%

**目標:** 100kgの飼料を最小コストで作る

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 各原料の使用量（kg）
@variable(model, a >= 0)  # 原料A
@variable(model, b >= 0)  # 原料B

# 総量制約
@constraint(model, a + b == 100)

# 栄養素制約（パーセンテージで計算）
@constraint(model, 0.30a + 0.20b >= 25)  # タンパク質最低
@constraint(model, 0.10a + 0.15b >= 12)  # 脂肪最低
@constraint(model, 0.05a + 0.10b <= 8)   # 繊維最大

# 目的関数（コスト最小化）
@objective(model, Min, 5a + 8b)

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最小コスト: \$", round(objective_value(model), digits=2))
println("\n配合:")
println("  原料A: ", round(value(a), digits=2), " kg")
println("  原料B: ", round(value(b), digits=2), " kg")

# 栄養素の確認
protein = 0.30 * value(a) + 0.20 * value(b)
fat = 0.10 * value(a) + 0.15 * value(b)
fiber = 0.05 * value(a) + 0.10 * value(b)
println("\n栄養素:")
println("  タンパク質: ", round(protein, digits=2), "% (最低25%)")
println("  脂肪: ", round(fat, digits=2), "% (最低12%)")
println("  繊維: ", round(fiber, digits=2), "% (最大8%)")

## 例題2: 輸送問題

### 問題

3つの工場から4つの倉庫へ製品を輸送します。

**供給量:** 工場1=100, 工場2=150, 工場3=120

**需要量:** 倉庫A=80, 倉庫B=90, 倉庫C=110, 倉庫D=90

**輸送コスト（\$/個）:**

|  | 倉庫A | 倉庫B | 倉庫C | 倉庫D |
|--|-------|-------|-------|-------|
| 工場1 | 8 | 10 | 6 | 7 |
| 工場2 | 9 | 12 | 13 | 7 |
| 工場3 | 14 | 9 | 16 | 5 |

**目標:** 総輸送コストを最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# データ定義
factories = ["F1", "F2", "F3"]
warehouses = ["WA", "WB", "WC", "WD"]

supply = Dict("F1" => 100, "F2" => 150, "F3" => 120)
demand = Dict("WA" => 80, "WB" => 90, "WC" => 110, "WD" => 90)

cost = Dict(
    ("F1", "WA") => 8,  ("F1", "WB") => 10, ("F1", "WC") => 6,  ("F1", "WD") => 7,
    ("F2", "WA") => 9,  ("F2", "WB") => 12, ("F2", "WC") => 13, ("F2", "WD") => 7,
    ("F3", "WA") => 14, ("F3", "WB") => 9,  ("F3", "WC") => 16, ("F3", "WD") => 5
)

# 決定変数：各ルートの輸送量
@variable(model, x[f in factories, w in warehouses] >= 0)

# 供給制約
for f in factories
    @constraint(model, sum(x[f, w] for w in warehouses) <= supply[f])
end

# 需要制約
for w in warehouses
    @constraint(model, sum(x[f, w] for f in factories) >= demand[w])
end

# 目的関数
@objective(model, Min, sum(cost[f, w] * x[f, w] for f in factories, w in warehouses))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最小輸送コスト: \$", round(objective_value(model), digits=2))
println("\n輸送計画:")
print("        ")
for w in warehouses
    print(lpad(w, 8))
end
println()
for f in factories
    print(f, ":     ")
    for w in warehouses
        print(lpad(round(Int, value(x[f, w])), 8))
    end
    println()
end

## 例題3: 投資ポートフォリオ問題

### 問題

3種類の投資先に\$100,000を配分します。

|  | 期待リターン | リスク |
|--|--------------|--------|
| 株式 | 12% | 高 |
| 債券 | 6% | 低 |
| 不動産 | 9% | 中 |

**制約：**
- 株式への投資は全体の50%以下
- 債券への投資は最低20%
- 不動産への投資は\$30,000以下

**目標:** 期待リターンを最大化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

total_investment = 100000

# 決定変数（投資額）
@variable(model, stock >= 0)       # 株式
@variable(model, bond >= 0)        # 債券
@variable(model, real_estate >= 0) # 不動産

# 総額制約
@constraint(model, stock + bond + real_estate == total_investment)

# 各資産の制約
@constraint(model, stock <= 0.5 * total_investment)      # 株式上限
@constraint(model, bond >= 0.2 * total_investment)       # 債券下限
@constraint(model, real_estate <= 30000)                 # 不動産上限

# 目的関数（期待リターン）
@objective(model, Max, 0.12 * stock + 0.06 * bond + 0.09 * real_estate)

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("期待リターン: \$", round(objective_value(model), digits=2))
println("\n投資配分:")
println("  株式: \$", round(value(stock), digits=2), 
        " (", round(value(stock)/total_investment*100, digits=1), "%)")
println("  債券: \$", round(value(bond), digits=2),
        " (", round(value(bond)/total_investment*100, digits=1), "%)")
println("  不動産: \$", round(value(real_estate), digits=2),
        " (", round(value(real_estate)/total_investment*100, digits=1), "%)")

## 例題4: 多期間生産計画問題

### 問題

ある工場で3ヶ月間の生産計画を立てます。

**月別需要予測:** 1月=100個, 2月=150個, 3月=120個

**コスト:**
- 通常生産: \$10/個（最大80個/月）
- 残業生産: \$15/個（最大40個/月）
- 在庫保管: \$2/個/月

**初期在庫:** 20個
**最終在庫:** 30個以上必要

**目標:** 総コストを最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

months = 1:3
demand = [100, 150, 120]
initial_inv = 20

# 決定変数
@variable(model, 0 <= regular[months] <= 80)   # 通常生産
@variable(model, 0 <= overtime[months] <= 40)  # 残業生産
@variable(model, inventory[months] >= 0)       # 在庫

# 在庫バランス制約
@constraint(model, initial_inv + regular[1] + overtime[1] - demand[1] == inventory[1])
for m in 2:3
    @constraint(model, inventory[m-1] + regular[m] + overtime[m] - demand[m] == inventory[m])
end

# 最終在庫制約
@constraint(model, inventory[3] >= 30)

# 目的関数
@objective(model, Min, sum(10*regular[m] + 15*overtime[m] + 2*inventory[m] for m in months))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("総コスト: \$", round(objective_value(model), digits=0))

println("\n月別生産計画:")
println("月    需要  通常  残業  在庫")
println("-" ^ 35)
for m in months
    println(m, "月   ", lpad(demand[m], 3), "   ",
            lpad(round(Int, value(regular[m])), 4), "  ",
            lpad(round(Int, value(overtime[m])), 4), "  ",
            lpad(round(Int, value(inventory[m])), 4))
end

## 例題5: ダイエット問題（栄養最適化）

### 問題

最小コストで必要な栄養素を摂取できる食事を計画します。

**食品データ（100gあたり）:**

|  | 価格 | カロリー | タンパク質 | 脂質 | 炭水化物 |
|--|------|----------|------------|------|----------|
| 鶏肉 | \$3.0 | 200 | 25g | 10g | 0g |
| 魚 | \$4.0 | 150 | 20g | 5g | 0g |
| 米 | \$1.0 | 350 | 7g | 1g | 77g |
| 野菜 | \$2.0 | 50 | 3g | 0g | 10g |
| 卵 | \$2.5 | 150 | 13g | 11g | 1g |

**1日の必要量:**
- カロリー: 1800-2200 kcal
- タンパク質: 50g以上
- 脂質: 30-70g
- 炭水化物: 200-300g

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 食品データ
foods = ["鶏肉", "魚", "米", "野菜", "卵"]
n_foods = length(foods)

price = [3.0, 4.0, 1.0, 2.0, 2.5]
calories = [200, 150, 350, 50, 150]
protein = [25, 20, 7, 3, 13]
fat = [10, 5, 1, 0, 11]
carbs = [0, 0, 77, 10, 1]

# 決定変数（100g単位）
@variable(model, x[1:n_foods] >= 0)

# 栄養素制約
@constraint(model, sum(calories[i] * x[i] for i in 1:n_foods) >= 1800)
@constraint(model, sum(calories[i] * x[i] for i in 1:n_foods) <= 2200)
@constraint(model, sum(protein[i] * x[i] for i in 1:n_foods) >= 50)
@constraint(model, sum(fat[i] * x[i] for i in 1:n_foods) >= 30)
@constraint(model, sum(fat[i] * x[i] for i in 1:n_foods) <= 70)
@constraint(model, sum(carbs[i] * x[i] for i in 1:n_foods) >= 200)
@constraint(model, sum(carbs[i] * x[i] for i in 1:n_foods) <= 300)

# 目的関数
@objective(model, Min, sum(price[i] * x[i] for i in 1:n_foods))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("1日の食費: \$", round(objective_value(model), digits=2))

println("\n食事計画（100g単位）:")
for i in 1:n_foods
    v = value(x[i])
    if v > 0.01
        println("  ", foods[i], ": ", round(v * 100, digits=0), "g")
    end
end

# 栄養素の計算
total_cal = sum(calories[i] * value(x[i]) for i in 1:n_foods)
total_pro = sum(protein[i] * value(x[i]) for i in 1:n_foods)
total_fat = sum(fat[i] * value(x[i]) for i in 1:n_foods)
total_carb = sum(carbs[i] * value(x[i]) for i in 1:n_foods)

println("\n栄養素の合計:")
println("  カロリー: ", round(total_cal, digits=0), " kcal (1800-2200)")
println("  タンパク質: ", round(total_pro, digits=1), "g (50以上)")
println("  脂質: ", round(total_fat, digits=1), "g (30-70)")
println("  炭水化物: ", round(total_carb, digits=1), "g (200-300)")

## 第3章のまとめ

この章で学んだこと：

1. **ブレンド問題**
   - 複数の原料を最適に配合
   - 品質制約と総量制約

2. **輸送問題**
   - 供給地から需要地への最適配送
   - 供給制約と需要制約

3. **ポートフォリオ問題**
   - 投資配分の最適化
   - リスク制約とリターン最大化

4. **多期間計画問題**
   - 時間軸を考慮した計画
   - 在庫バランス制約

5. **ダイエット問題**
   - 複数の制約を同時に満たす
   - 実用的な応用例

---

**次の章:** 混合整数線形計画法を学びます！